In [64]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import urllib
import pandas as pd

DRIVER_PATH = './chromedriver.exe'
BASE_URL = 'https://map.naver.com/v5/search/'

def openTargetBrowser(keyword):
    
    # 키워드를 URL 형식으로 변환
    encText = urllib.parse.quote(keyword)
    url = BASE_URL + encText
    
    # selenium 브라우저로 해당 URL 접속
    browser = webdriver.Chrome(DRIVER_PATH)
    time.sleep(2)
    browser.get(url)

    # 필요한 정보가 있는 iframe 페이지에 name 속성명을 통한 접근
    browser.switch_to.frame('searchIframe')
    time.sleep(2)
    
    return browser

def scrollingPage(browser):
    # 스크롤을 하면 추가되는 <li> 항목을 모두 가져오기 위한 div 스크롤 조작
    container = browser.find_element('id','_pcmap_list_scroll_container')
    init = 3000
    
    for i in range(3):
        init += i*1000
        browser.execute_script(f"arguments[0].scrollBy(0, {init})", container)                                
        time.sleep(2)
        
    return browser

def splitData(kind, info):
    kindInfo = {'blog':'블로그리뷰','visit':'방문자리뷰'}
    # 블로그리뷰 를 기준으로 split한 리스트의 길이가 1이면
    # 블로그리뷰에 대한 정보가 없는 것이므로 0을 저장
    if len(info.split( kindInfo[kind] ))== 1:
        return None, info
    else:
        info = info.split( kindInfo[kind] )
        num = int(info[-1].replace(',',''))
        
        # split된 텍스트 중 나머지 정보가 담긴 텍스트로 이동
        return num, info[0]

def splitReviewNum(info):
    if info:
        info = info.text
        
        blogNum, nextInfo = splitData("blog", info)
        visitNum, usedInfo = splitData("visit", nextInfo)
    else:
        blogNum = None
        visitNum = None
        
    return visitNum, blogNum

def isLastPage(browser):
    # 이전 페이지, 5개의 페이지 번호, 다음 페이지 버튼으로 구성된
    # 7개의 <a> 태그 중 다음 페이지 버튼인 마지막 <a> 태그 선택
    nextBtn = browser.find_elements(By.CSS_SELECTOR,'div._2ky45 a')[-1]

    # 해당 버튼의 class명 확인
    btnClass = nextBtn.get_attribute("class")
    print(btnClass)

    # 클래스명이 "_2bgjK "이면 (공백 포함 주의 !!)
    # 마지막 페이지가 아니므로 다음 페이지로 이동
    if btnClass == "_2bgjK ":
        print("this is not last page.")
        nextBtn.click()
        time.sleep(2)
        return False

    # class명이 "_2bgjK _34lTS"이면 마지막 페이지이므로 반복문 종료
    else:
        return True

def navigatingPage(browser):
    # 데이터를 담을 리스트와 각 항목에 대한 접근을 확인하기 위한 cnt 변수 생성
    outList = []
    cnt = 0
    
    while True: 
        browser = scrollingPage(browser)
        
        # 스크롤 된 페이지 소스를 bs4 객체로 변환
        htmlStr = browser.page_source
        bs = BeautifulSoup(htmlStr,'html.parser')
        
        targetDiv = bs.select_one('div#_pcmap_list_scroll_container')
        targets = targetDiv.select("li[class='_1EKsQ _12tNp']")
        
        # 페이지당 50개인 <li> 항목을 모두 가져왔는지 확인 
        print(len(targets))

        for t in targets:
            
            # 접근 중인 항목 번호 
            cnt += 1
            print(cnt)
            
            # 점포명
            name = t.select_one('span.place_bluelink').text

            # 별점 (별점이 없으면 None을 저장)
            score = t.select_one('em')
            if score:
                score = float(score.text)
            else:
                score = None
            
            # 썸네일 이미지
            img = t.select_one('div.cb7hz')
            if img:
                img = img['style'].split('"')[1]
            else:
                img = None
            
            # 영업 상태, 별점, 리뷰 등이 담겨있는 div 태그 텍스트
            containerInfo = t.select_one('div._17H46')
            visitNum, blogNum = splitReviewNum(containerInfo)
            
            outList.append([name, score, img, visitNum, blogNum])

        # 탐색이 끝난 후 다음 페이지로 이동
        if isLastPage(browser):
            break
            
    return outList

def ratingData(result):
    df = pd.DataFrame(result,columns=['점포명','별점','이미지','방문자 리뷰 수', '블로그 리뷰 수'])

    # 별점이나 방문자 리뷰 수, 블로그 리뷰 수 중 하나라도 결측값이 있는 레코드 삭제
    df = df.dropna(how='any', subset=['별점','방문자 리뷰 수','블로그 리뷰 수'])
    
    # Pandas의 qcut() 함수를 이용해 리뷰 수 분포에 따른 구간 산출
    # 동점이 많은 낮은 구간에서 구간의 중복이 발생하는 것에 대해 rank 함수를 사용한 순위 부여
    # 첫번째 값 (method='first') : 동점 관측치 중에서 데이터 상에서 먼저 나타나는 관측치부터 순위 부여를 부여하는 rank 함수의 옵션
    df['방문자 리뷰 구간'] = pd.qcut(df['방문자 리뷰 수'].rank(method = 'first'),10, labels=False)
    df['블로그 리뷰 구간'] = pd.qcut(df['블로그 리뷰 수'].rank(method = 'first'),10, labels=False)

    display(df.describe())
    
    # 방문자 리뷰와 블로그 리뷰의 중요도에 따른 가중치 부여
    visitWeight = 0.2
    blogWeight = 0.8
    totalScore = df['별점'] + (((df['방문자 리뷰 구간'] + 1) / 10) * visitWeight) + (((df['블로그 리뷰 구간'] + 1) / 10) * blogWeight) + 4
    df['평점'] = round(totalScore, 2)
    
    # 데이터형 정제
    df = df.astype({
        '방문자 리뷰 수' : 'int',
        '블로그 리뷰 수' : 'int',
    })
    
    # 평점 기준 상위 5개 데이터 확인
    display(df.nlargest(5,'평점', keep='first'))
    
    return df

    
def collectFamousStore(kor,eng):
    # 입력받은 지하철역 키워드 생성
    print(kor, eng)
    keyword = kor + "역맛집"
    
    targetBrowser = openTargetBrowser(keyword)
    
    targetList = navigatingPage(targetBrowser)
    
    resultDF = ratingData(targetList)

    # 엑셀에서 한글이 깨지지 않는 'utf-8-sig' 인코딩으로 csv 파일 추출

    resultDF.to_csv(f"{eng}.csv",encoding='utf-8-sig')

if __name__ == "__main__":
    
# 지하철역명 입력 후 collectFamousStore 함수로 전달 
#     collectFamousStore(input())

    # 부산교통공사_도시철도역정보_20211020 파일에서 1호선 역명과, 영문명 추출
    df = pd.read_csv('./부산교통공사_도시철도역정보_20211020.csv', encoding='cp949')
    df = df.loc[df['역코드'] < 200,['역명','영문']]
    df['역명'] = df['역명'].str.replace('역','').str.replace(r"\(.*\)","")
    df['영문'] = df['영문'].str.replace(r"[ .]","")
    
    stationList = df.to_numpy().tolist()  
    dirList = os.listdir()
    
    for kor,eng in stationList:
        # 현재 경로 내에 이미 크롤링한 파일이 있으면 continue
        if (eng + '.csv') in dirList: continue
        collectFamousStore(kor,eng)

C:\Users\pnu\AppData\Local\Temp\ipykernel_1880\174677803.py:195: FutureWarning: The default value of regex will change from True to False in a future version.
  df['역명'] = df['역명'].str.replace('역','').str.replace(r"\(.*\)","")
C:\Users\pnu\AppData\Local\Temp\ipykernel_1880\174677803.py:196: FutureWarning: The default value of regex will change from True to False in a future version.
  df['영문'] = df['영문'].str.replace(r"[ .]","")
C:\Users\pnu\AppData\Local\Temp\ipykernel_1880\174677803.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(DRIVER_PATH)


토성 Toseong
46
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
_2bgjK 
this is not last page.
50
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
_2bgjK 
this is not last page.
50
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
_2bgjK 
this is not last page.
50
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
_2bgjK 
this is not last page.
50
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
24

,별점,방문자 리뷰 수,블로그 리뷰 수,방문자 리뷰 구간,블로그 리뷰 구간
count,244.000000,244.000000,244.000000,244.000000,244.000000
mean,4.520902,410.766393,145.377049,4.500000,4.500000
std,0.163289,707.071372,220.402786,2.886751,2.886751
min,4.030000,14.000000,1.000000,0.000000,0.000000
25%,4.430000,122.500000,42.500000,2.000000,2.000000
50%,4.500000,203.500000,94.000000,4.500000,4.500000
75%,4.602500,389.500000,157.000000,7.000000,7.000000
max,5.000000,5657.000000,2001.000000,9.000000,9.000000


,점포명,별점,이미지,방문자 리뷰 수,블로그 리뷰 수,방문자 리뷰 구간,블로그 리뷰 구간,평점
16,여기제주도,4.86,None,434,753,7,9,9.82
167,하이포,4.83,None,530,185,8,8,9.73
31,귀인,4.74,None,326,434,6,9,9.68
13,띠리오,4.69,None,459,485,8,9,9.67
64,보느파티쓰리,4.67,None,1008,428,9,9,9.67


서면 Seomyeon


...

노포 Nopo


C:\Users\pnu\AppData\Local\Temp\ipykernel_1880\174677803.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(DRIVER_PATH)


50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
_2bgjK 
this is not last page.
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
_2bgjK 
this is not last page.
50
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
_2bgjK 
this is not last page.
50
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
_2bgjK 
this is not last page.
50
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
2

,별점,방문자 리뷰 수,블로그 리뷰 수,방문자 리뷰 구간,블로그 리뷰 구간
count,130.000000,130.000000,130.000000,130.000000,130.000000
mean,4.449462,188.400000,29.238462,4.500000,4.500000
std,0.202887,238.799829,66.601884,2.883393,2.883393
min,3.790000,10.000000,1.000000,0.000000,0.000000
25%,4.340000,48.500000,2.250000,2.000000,2.000000
50%,4.470000,103.500000,8.000000,4.500000,4.500000
75%,4.580000,212.750000,25.750000,7.000000,7.000000
max,4.900000,1600.000000,593.000000,9.000000,9.000000


,점포명,별점,이미지,방문자 리뷰 수,블로그 리뷰 수,방문자 리뷰 구간,블로그 리뷰 구간,평점
1,우렁각시와꽃길,4.88,https://search.pstatic.net/common/?autoRotate=...,343,41,8,8,9.78
184,프롬엘키친,4.76,None,157,161,6,9,9.70
35,디위드,4.81,None,63,94,3,9,9.69
3,금샘다방,4.68,https://search.pstatic.net/common/?autoRotate=...,476,593,9,9,9.68
6,원조동촌칼국수,4.61,None,548,136,9,9,9.61
